In [1]:
from PIL import Image
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda', index=0)

In [3]:
class ImageDataset(torch.utils.data.Dataset):

    def __init__(self, image_files, text, processor, num_labels):
        self.image_files = image_files
        self.text = text
        self.processor = processor
        self.num_labels = num_labels

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        text = self.text
        image = self.image_files[idx]['img']
        label = self.image_files[idx]['label']
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        encoding = self.processor(image, text, padding="max_length", truncation=True, return_tensors="pt")

        # remove batch dimension
        for k,v in encoding.items():
            encoding[k] = v.squeeze()
        targets = torch.zeros(self.num_labels)
        targets[label] = 1
        encoding["labels"] = targets

        return encoding

In [4]:
!pip install datasets
from datasets import load_dataset

cache_dir='./cache'
#datasets = load_dataset('Maysee/tiny-imagenet', cache_dir=cache_dir)
datasets = load_dataset('cifar10', cache_dir=cache_dir)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.4 MB/s eta 0:00:00


Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/datasets/features/image.py:325: UserWarning: Downcasting array dtype uint8 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /content/cache/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.4 MB/s eta 0:00:00


In [6]:
from transformers import ViltProcessor, BertTokenizer

label_list = datasets["train"].features["label"].names
num_labels = len(label_list)

config = BertTokenizer.from_pretrained("bert-base-uncased",padding="max_length",max_length=4)
config.id2label = {str(i): label for i, label in enumerate(label_list)}
config.label2id = {label: str(i) for i, label in enumerate(label_list)}
config.num_labels = num_labels

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

train_dataset = ImageDataset(image_files=datasets["train"], text="What is this image?", processor=processor, num_labels=num_labels)
test_dataset = ImageDataset(image_files=datasets["test"], text="What is this image?", processor=processor, num_labels=num_labels)

In [7]:
from torch.utils.data import DataLoader
import torchvision.transforms as T
transform = T.Resize((224,224))
def collate_fn(batch):
  input_ids = [item['input_ids'] for item in batch]
  pixel_values = [item['pixel_values'] for item in batch]
  attention_mask = [item['attention_mask'] for item in batch]
  token_type_ids = [item['token_type_ids'] for item in batch]
  labels = [item['labels'] for item in batch]

  # create padded pixel values and corresponding pixel mask
  encoding = processor.feature_extractor.pad_and_create_pixel_mask(pixel_values, return_tensors="pt")

  # create new batch
  batch = {}
  batch['input_ids'] = torch.stack(input_ids)
  batch['attention_mask'] = torch.stack(attention_mask)
  batch['token_type_ids'] = torch.stack(token_type_ids)
  batch['pixel_values'] = transform(encoding['pixel_values'])
  batch['pixel_mask'] = encoding['pixel_mask']
  batch['labels'] = torch.stack(labels)

  return batch

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=64, shuffle=True)

In [ ]:
type(datasets)

datasets.dataset_dict.DatasetDict

In [8]:
!pip install torchmultimodal-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.6 MB/s eta 0:00:00
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=50ba81ca32d010baf4b29d188925623d4b9c4c4f238292c42e5ba2ef00df14eb
  Stored in directory: /root/.cache/pip/wheels/c1/13/6d/441d8f2af76ee6d2a3e67eebb1d0c556fefcee0a8b32266a8e
Successfully built iopath


In [9]:
from torchmultimodal.models.flava.model import flava_model_for_classification
# vocab = datasets["train"].features["label"].names
model_1 = flava_model_for_classification(num_classes=10)

flava_for_pretraining_unified_text_encoder.pt: 1.43GB [00:35, 40.7MB/s]                            


In [10]:
model_1

FLAVAForClassification(
  (model): FLAVAModel(
    (image_encoder): ImageTransformer(
      (embeddings): ImageEmbeddings(
        (patch_embeddings): PatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer): ModuleList(
          (0): TransformerEncoderLayer(
            (attention): MultiHeadAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (output): Linear(in_features=768, out_features=768, bias=True)
              (attn): SelfAttention()
            )
            (attention_dropout): Dropout(p=0.0, inplace=False)
            (feedforward): MLP(
              (model): Sequential(
                (0): Linear(in_features=768, out_fe

In [11]:
from torch import nn
from torch.utils.data import DataLoader

model_1.to(device)
optimizer = torch.optim.AdamW(model_1.parameters())

for param in model_1.parameters():
    param.requires_grad = False

for name, param in model_1.named_parameters():
    if 'classifier' in name:
        param.requires_grad = True

epochs = 1
a = 0
b = 0
for _ in range(epochs):
  for idx, batch in enumerate(train_dataloader):
    optimizer.zero_grad()
    out = model_1(text = batch["input_ids"].to(device), image = batch["pixel_values"].to(device), labels = batch["labels"].to(device))
    loss = out.loss
    preds = torch.argmax(out['logits'], dim=1)
    target = torch.argmax(batch['labels'].to(device), dim=1)
    correct = torch.sum(preds==target).item()
    acc = (correct * 100) / target.size(0)
    loss.backward()
    optimizer.step()
    print(f"Loss at step {idx} = {loss}  Accuracy = {acc}")


/usr/local/lib/python3.9/dist-packages/transformers/models/vilt/processing_vilt.py:142: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/vilt/image_processing_vilt.py:357: FutureWarning: This method is deprecated and will be removed in v4.26.0. Please use pad instead.
  warnings.warn(


Loss at step 0 = 2.3039920330047607  Accuracy = 7.8125
Loss at step 1 = 2.2975077629089355  Accuracy = 18.75
Loss at step 2 = 2.290396213531494  Accuracy = 26.5625
Loss at step 3 = 2.272592544555664  Accuracy = 23.4375
Loss at step 4 = 2.276644229888916  Accuracy = 23.4375
Loss at step 5 = 2.258100748062134  Accuracy = 25.0
Loss at step 6 = 2.286461114883423  Accuracy = 15.625
Loss at step 7 = 2.27683162689209  Accuracy = 10.9375
Loss at step 8 = 2.249321937561035  Accuracy = 17.1875
Loss at step 9 = 2.2512266635894775  Accuracy = 9.375
Loss at step 10 = 2.207352876663208  Accuracy = 29.6875
Loss at step 11 = 2.2554972171783447  Accuracy = 18.75
Loss at step 12 = 2.2115511894226074  Accuracy = 25.0
Loss at step 13 = 2.2036685943603516  Accuracy = 28.125
Loss at step 14 = 2.2005858421325684  Accuracy = 25.0
Loss at step 15 = 2.199254035949707  Accuracy = 29.6875
Loss at step 16 = 2.1873340606689453  Accuracy = 32.8125
Loss at step 17 = 2.179293632507324  Accuracy = 37.5
Loss at step 18 

In [12]:
correct_test = 0
total_test = 0
for idx, batch in enumerate(test_dataloader):
    out = model_1(text = batch["input_ids"].to(device), image = batch["pixel_values"].to(device), labels = batch["labels"].to(device))
    loss = out.loss
    preds = torch.argmax(out['logits'], dim=1)
    target = torch.argmax(batch['labels'].to(device), dim=1)
    batch_acc = torch.sum(preds==target).item() * 100 / target.size(0)
    print(f"Test Accuracy for batch = {batch_acc}")
    correct_test += torch.sum(preds==target).item()
    total_test += target.size(0)

/usr/local/lib/python3.9/dist-packages/transformers/models/vilt/processing_vilt.py:142: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/vilt/image_processing_vilt.py:357: FutureWarning: This method is deprecated and will be removed in v4.26.0. Please use pad instead.
  warnings.warn(


Test Accuracy for batch = 93.75
Test Accuracy for batch = 92.1875
Test Accuracy for batch = 93.75
Test Accuracy for batch = 93.75
Test Accuracy for batch = 93.75
Test Accuracy for batch = 93.75
Test Accuracy for batch = 90.625
Test Accuracy for batch = 93.75
Test Accuracy for batch = 93.75
Test Accuracy for batch = 93.75
Test Accuracy for batch = 95.3125
Test Accuracy for batch = 89.0625
Test Accuracy for batch = 95.3125
Test Accuracy for batch = 95.3125
Test Accuracy for batch = 93.75
Test Accuracy for batch = 95.3125
Test Accuracy for batch = 96.875
Test Accuracy for batch = 95.3125
Test Accuracy for batch = 93.75
Test Accuracy for batch = 90.625
Test Accuracy for batch = 93.75
Test Accuracy for batch = 85.9375
Test Accuracy for batch = 93.75
Test Accuracy for batch = 96.875
Test Accuracy for batch = 90.625
Test Accuracy for batch = 95.3125
Test Accuracy for batch = 90.625
Test Accuracy for batch = 92.1875
Test Accuracy for batch = 90.625
Test Accuracy for batch = 92.1875
Test Accura

In [13]:
print('Training accuracy: ' + str(correct_test * 100/total_test))

Training accuracy: 92.74
